In [ ]:
with open("C_example.txt", "r") as file:
	C_example = file.read()
with open("P_example.txt", "r") as file:
  P_example = file.read()
def init_template(type, example):
    return {
    "custom_id": None, 
    "method": "POST", 
    "url": "/v1/chat/completions",
    "body": {"model": "gpt-4.1", 
             "messages": 
                 [{"role": "system", "content": f"""
                   You are a helpful Ontology engineer. 
                   Generate a {type} description including information of axioms. 
                   The description should be concise and informative, providing a clear understanding of the {type}'s purpose and characteristics.
                   Don't generate unnecessary text. Just generate {type} description only."""},
                   {"role": "user", "content": f"""
                   Generate a {type} description including information of axioms. 
                   If there is no description, generate a description based on the axiom."""},
                   {"role": "user", "content": "For example, "+example},
                   {"role": "user", "content": "Now, generate the description using class axiom below."}
                  ],
             "max_tokens": 1024 
             }
    }

In [ ]:
import os
directory_path = "Ontology_description"
ontology_list = {file.split("_")[0]: {"description":os.path.join(directory_path, file)} for file in os.listdir(directory_path)}
print(ontology_list.keys())

In [ ]:
import json
from copy import deepcopy
import random
data_num = 0
batches = []
for ontology in ontology_list:
    with open("processed_type2.json", "r") as f:
        temp_axiom=json.load(f)[ontology]
    for cls in temp_axiom["classes"]:
        temp = deepcopy(init_template("class", C_example))
        temp["body"]["messages"].append({"role": "user", "content": "Class name: "+cls})
        temp["custom_id"] = ontology+"_"+cls
        temp["body"]["messages"].append({"role": "user", "content": "Axiom: "+str(temp_axiom["classes"][cls]["axiom"])})
        temp["body"]["messages"].append({"role": "user", "content": "Generated description: "})
        batches.append(temp)
    for prop in temp_axiom["properties"]:
        temp = deepcopy(init_template("property", P_example))
        temp["body"]["messages"].append({"role": "user", "content": "Property name: "+prop})
        temp["custom_id"] = ontology+"_"+prop
        temp["body"]["messages"].append({"role": "user", "content": "Axiom: "+str(temp_axiom["properties"][prop]["axiom"])})
        temp["body"]["messages"].append({"role": "user", "content": "Generated description: "})
        batches.append(temp)
    data_num += len(batches)
# Save the batches to a JSONL file
with open("Batchinput.jsonl", 'w', encoding='utf-8') as file:
    for item in batches:
        json_string = json.dumps(item, ensure_ascii=False)
        file.write(json_string + '\n')
    print(f"Ontology name: {ontology}, Number of batches: {len(batches)}")
print(f"Total number of batches: {data_num}")
    
    

In [ ]:
from openai import OpenAI
import time
client = OpenAI(api_key="")


with open("Batchinput.jsonl", "rb") as file:
  batch_input_file = client.files.create(
    file=file, 
    purpose="batch"
  )
batch_input_file_id = batch_input_file.id

# Create a batch job
client.batches.create(
  input_file_id=batch_input_file_id,
  endpoint="/v1/chat/completions",
  completion_window="24h", 
        )


Batch(id='batch_681b4770ef9481908d94253e31aa5e7b', completion_window='24h', created_at=1746618224, endpoint='/v1/chat/completions', input_file_id='file-BKQvPTHkNWQCVBXKwp4kpc', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1746704624, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
import os
import json

jsonl_files = ["Batchoutput.jsonl"]

for jsonl_file in jsonl_files:
    loaded_data = []
    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            temp_data = json.loads(line)
            temp_out = {"class": temp_data["custom_id"].split("_",1)[1], "description": temp_data["response"]["body"]["choices"][0]["message"]["content"]}
            loaded_data.append(temp_out)
    # Save the loaded data to a new JSONL file
    with open("Generated_description.jsonl", "w", encoding="utf-8") as f:
        for item in loaded_data:
            json_string = json.dumps(item, ensure_ascii=False)
            f.write(json_string + '\n')

In [6]:
import json
with open("processed_type2.json", "r", encoding="utf-8") as outfile:
    total_type2 = json.load(outfile)
with open("Generated_description.jsonl", "r", encoding="utf-8") as f:
    temp_desc = {}
    for line in f:
        loadfile = json.loads(line)
        temp_desc[loadfile["class"]] = loadfile["description"]
for ontology in total_type2:
    for cls in total_type2[ontology]["classes"]:
        total_type2[ontology]["classes"][cls]["description"] = temp_desc[cls]
    for prop in total_type2[ontology]["properties"]:
        total_type2[ontology]["properties"][prop]["description"] = temp_desc[prop]
with open("type2_description_update.json", "w", encoding="utf-8") as outfile:
    json.dump(total_type2, outfile, ensure_ascii=False, indent=4)

In [7]:
import json

# List of axiom predicates to look for
axiom_relations = [
    "subClassOf", "equivalentClass", "propertyRestrictions",
    "disjointWith", "subPropertyOf", "domain", "range",
    "characteristics", "inverseOf"
]

def restore_removed_axioms(input_path='type2_description_update.json', output_path='Final_type2.json'):
    # 1. Load the processed data
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. Traverse each ontology, classes and properties
    for ontology in data.values():
        for section in ('classes', 'properties'):
            for name, info in ontology.get(section, {}).items():
                removed = info.get('removed axiom')
                if not removed:
                    continue

                # find predicate
                predicate = next((rel for rel in axiom_relations if f" {rel} " in removed), None)
                if predicate is None:
                    continue

                # split into subject and expression
                _, expr = removed.split(f" {predicate} ", 1)
                expr = expr.strip()

                # restore into info['axiom'][predicate]
                ax = info.setdefault('axiom', {})
                current = ax.setdefault(predicate, [])
                if isinstance(current, list) and expr not in current:
                    current.append(expr)

    # 3. Write out the restored file
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# Example usage:
restore_removed_axioms(
    input_path='type2_description_update.json',
    output_path='Final_type2.json'
)
